# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [19]:
import pandas as pd
import statsmodels.api as sm
import patsy as pt
import numpy as np

from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor




In [20]:
df = pd.read_csv('previsao_de_renda.csv')

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   index                  15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12466 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  mau                    15000 non-null  bool   
 15  re

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [22]:
# Listar todas as colunas booleanas
bool_columns = df.select_dtypes(include='bool').columns

# Converter as colunas booleanas para 0s e 1s
df[bool_columns] = df[bool_columns].astype(int)

# Verificar o novo DataFrame para garantir que as colunas booleanas foram convertidas
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   index                  15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  int32  
 5   posse_de_imovel        15000 non-null  int32  
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12466 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  mau                    15000 non-null  int32  
 15  re

In [32]:
df = pd.get_dummies(df, drop_first=True)
columns_list = ['posse_de_veiculo', 'posse_de_imovel', 'qtd_filhos', 'idade', 'tempo_emprego', 'qt_pessoas_residencia', 'mau', 'log_renda', 'sexo_M', 'tipo_renda_Bolsista', 'tipo_renda_Empresário', 'tipo_renda_Pensionista', 'tipo_renda_Servidor público', 'educacao_Pós graduação', 'educacao_Secundário', 'educacao_Superior completo', 'educacao_Superior incompleto', 'estado_civil_Separado', 'estado_civil_Solteiro', 'estado_civil_União', 'estado_civil_Viúvo', 'tipo_residencia_Casa', 'tipo_residencia_Com os pais', 'tipo_residencia_Comunitário', 'tipo_residencia_Estúdio', 'tipo_residencia_Governamental']

X = df[['posse_de_veiculo', 'posse_de_imovel', 'qtd_filhos', 'idade', 'tempo_emprego', 'qt_pessoas_residencia', 'mau', 'sexo_M', 'tipo_renda_Bolsista', 'tipo_renda_Empresário', 'tipo_renda_Pensionista', 'tipo_renda_Servidor público', 'educacao_Pós graduação', 'educacao_Secundário', 'educacao_Superior completo', 'educacao_Superior incompleto', 'estado_civil_Separado', 'estado_civil_Solteiro', 'estado_civil_União', 'estado_civil_Viúvo', 'tipo_residencia_Casa', 'tipo_residencia_Com os pais', 'tipo_residencia_Comunitário', 'tipo_residencia_Estúdio', 'tipo_residencia_Governamental']] # Features
y = df['renda']  # Target variable

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [33]:
mean_tempo_emprego = df['tempo_emprego'].mean()

df['tempo_emprego'].fillna(mean_tempo_emprego, inplace=True)
df = pd.get_dummies(df,drop_first=True)
df=df.astype(int)

In [34]:
df.isna().sum()

Unnamed: 0                       0
index                            0
posse_de_veiculo                 0
posse_de_imovel                  0
qtd_filhos                       0
idade                            0
tempo_emprego                    0
qt_pessoas_residencia            0
mau                              0
renda                            0
data_ref_2015-02-01              0
data_ref_2015-03-01              0
data_ref_2015-04-01              0
data_ref_2015-05-01              0
data_ref_2015-06-01              0
data_ref_2015-07-01              0
data_ref_2015-08-01              0
data_ref_2015-09-01              0
data_ref_2015-10-01              0
data_ref_2015-11-01              0
data_ref_2015-12-01              0
data_ref_2016-01-01              0
data_ref_2016-02-01              0
data_ref_2016-03-01              0
sexo_M                           0
tipo_renda_Bolsista              0
tipo_renda_Empresário            0
tipo_renda_Pensionista           0
tipo_renda_Servidor 

In [35]:
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
best_alpha = None
best_r2 = float('-inf')

for alpha in alphas:
    ridge_model = Ridge(alpha=alpha)
    ridge_model.fit(X_train, y_train)
    y_pred = ridge_model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    
    if r2 > best_r2:
        best_r2 = r2
        best_alpha = alpha

print(f"Best Ridge model alpha: {best_alpha}, R^2: {best_r2}")

Best Ridge model alpha: 0.1, R^2: 0.2353983762009274


In [37]:
best_alpha_lasso = None
best_r2_lasso = float('-inf')

for alpha in alphas:
    lasso_model = Lasso(alpha=alpha)
    lasso_model.fit(X_train, y_train)
    y_pred_lasso = lasso_model.predict(X_test)
    r2_lasso = r2_score(y_test, y_pred_lasso)
    
    if r2_lasso > best_r2_lasso:
        best_r2_lasso = r2_lasso
        best_alpha_lasso = alpha

print(f"Best LASSO model alpha: {best_alpha_lasso}, R^2: {best_r2_lasso}");


c:\Users\coimb\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:1474: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  return fit_method(estimator, *args, **kwargs)
c:\Users\coimb\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
c:\Users\coimb\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.326e+10, tolerance: 2.368e+07 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers i

Best LASSO model alpha: 0.1, R^2: 0.23543631075005023


c:\Users\coimb\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.495e+10, tolerance: 2.368e+07
  model = cd_fast.enet_coordinate_descent(


In [39]:
# Function for backward elimination
def backward_elimination(X, y, significance_level=0.05):
    X = sm.add_constant(X)  # Add constant term
    model = sm.OLS(y, X).fit()
    
    while True:
        p_values = model.pvalues
        max_p_value = max(p_values)
        
        if max_p_value > significance_level:
            excluded_feature = p_values.idxmax()
            X = X.drop(columns=[excluded_feature])
            model = sm.OLS(y, X).fit()
        else:
            break
    
    return model

final_model = backward_elimination(X_train, y_train)
print(final_model.summary())

                                 OLS Regression Results                                
Dep. Variable:                  renda   R-squared (uncentered):                -603.442
Model:                            OLS   Adj. R-squared (uncentered):           -603.926
Method:                 Least Squares   F-statistic:                             -1247.
Date:                Thu, 26 Sep 2024   Prob (F-statistic):                        1.00
Time:                        01:13:02   Log-Likelihood:                     -1.0948e+05
No. Observations:               11250   AIC:                                  2.190e+05
Df Residuals:                   11241   BIC:                                  2.190e+05
Df Model:                           9                                                  
Covariance Type:            nonrobust                                                  
                                  coef    std err          t      P>|t|      [0.025      0.975]
------------------------

In [40]:
# Ridge coefficients
ridge_coeffs = ridge_model.coef_

# LASSO coefficients
lasso_coeffs = lasso_model.coef_

# Summary of coefficients
comparison = pd.DataFrame({
    'Ridge': ridge_coeffs,
    'LASSO': lasso_coeffs
})
print(comparison)

          Ridge        LASSO
0      2.727010     2.453407
1    386.263571   385.975822
2   -608.797584  -435.957194
3     31.310550    31.295576
4    290.896917   290.855086
5    671.377619   498.610152
6    325.249677   325.534907
7   2286.452678  2285.838213
8   -643.731254  -378.395510
9    981.846133   981.408482
10  -843.186953  -843.292671
11   567.445308   566.096085
12  2890.547836  2796.985461
13   242.999485   214.193921
14   748.620482   719.249483
15   250.231609   219.149538
16   641.331526   466.340067
17   649.903420   478.484221
18  -126.493509  -126.471988
19   599.401842   424.398216
20  -575.473310  -537.813579
21  -747.839740  -708.418910
22 -1162.781894 -1101.663678
23 -1274.959434 -1222.980281
24 -1014.138164  -974.014562


In [41]:
tree_model = DecisionTreeRegressor()
tree_model.fit(X_train, y_train)
y_pred_tree = tree_model.predict(X_test)
r2_tree = r2_score(y_test, y_pred_tree)

print(f"Decision Tree R^2: {r2_tree}")

Decision Tree R^2: 0.20684624114446626
